In [1]:
# Parameters
STUDY_START_DATE = "2024-01-01"
STUDY_END_DATE = "2024-12-31"


Load in necessary libraries 

In [2]:

library(DBI)
library(RSQLite)
library(gtsummary)
library(here)
library(reticulate)
library(officer)
library(flextable)


here() starts at /home/jovyan/workspace




Attaching package: ‘flextable’




The following object is masked from ‘package:gtsummary’:

    continuous_summary




In order to load in the config file, which has the file paths listed for artifact (tables listings and figures) output, we need to setup a python virtual environment 

In the terminal run (only need to run once when setting up R session):
`uv venv -p 3.11` 

In the R console (only need to run once when setting up R session)
`reticulate::use_virtualenv("./.venv")`

Continue setting up reticulate and load in config variables 

In [3]:

config = import_from_path(module = "config", path = here("source", "notebooks", "helpers"))
output_file_path = config$OUTPUT_ARTIFACT_PATH


Warning message in poetry_config(required_module):
“This project appears to use Poetry for Python dependency management.
However, the 'poetry' command line tool is not available.
reticulate will be unable to activate this project.
Please ensure that 'poetry' is available on the PATH.”


Error in python_config_impl(python) : 
  Error running '/home/jovyan/workspace/venv/bin/python': No such file.
The Python installation used to create the virtualenv has been moved or removed:
  '/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/bin'


Load in data from python step 

In [4]:

conn = dbConnect(RSQLite::SQLite(), "sample.sqlite")

patients = dbGetQuery(conn, "SELECT * FROM filtered_patients")


Create output table using tbl_summary 

In [5]:
output_table = patients |>
  select(AGE, GENDER, RACE, ETHNICITY, IS_SMOKER, HOSPITALIZATION_COUNT) |>
  tbl_summary() |>
  #convert to flex table for officer output 
  as_flex_table()


Use officer to create output word file 

In [6]:

output_title = 'Table 1. Patient characteristics'

output_doc = read_docx() |>
  body_add(output_title) |>
  body_add_flextable(output_table) 

print(output_doc, file.path(output_file_path, 'patient_characteristics_tables.docx'))
